# Previous function for storing nodes efficiently

In [ ]:
def matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += (10**i)*matrice[i]
    return int(score)

In [ ]:
def simplified_matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += i*matrice[i]
    return int(score)

# Depth search

Let's use a function that computes a value for each matrix so that it will be easy to store matrices.

This function shouuld be bi-directional: a number is associated to only one matrix and viceversa.

In [ ]:
def matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += (10**i)*matrice[i]
    return int(score)


def simplified_matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += i*matrice[i]
    return int(score)

In [ ]:
def depth_limited_search(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int) -> list[action] or None:
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # # Backtrack if depth limit reached
        if depth >= max_depth:
            continue
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            
            # Check if the next state has already been visited in the current path
            if matrix_score(next_state) not in visited:
                # Add the new state and path to stack, increase depth
                stack.append((next_state, path + [act], depth + 1))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(matrix_score(current_state))
    
    return None  # Return None if no solution is found within depth limit

# Iterative Deepening Depth-First Search (IDDFS) wrapper function
def iterative_deepening_dfs(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int = 50) -> list[action] or None:
    for depth in range(1, max_depth + 1):
        result = depth_limited_search(initial_state, final_state, depth)
        if result is not None:
            return result
    return None


# Updated Depth search


At each iteration, among all the possible we choose the one with the minimum manahattan distance to the test goal. In case there is no state with less manhattan distance compared to the current we select the oen that is the "best" among the possibilities. In case they are the same we use the last one.

In [ ]:
def update_depth(initial_state: np.ndarray, final_state: np.ndarray) -> list or None:
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and calculate Manhattan distances for all possible moves
        successors = []
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = matrix_score(next_state)
            
            # Check if the next state has already been visited in the current path
            if next_score not in visited:
                # Calculate the Manhattan distance from the goal
                manhattan_dist = manhattan_distance(next_state, final_state)
                # Append (next_state, path + [act], depth + 1, manhattan_dist) to successors list
                successors.append((next_state, path + [act], depth + 1, manhattan_dist))
        
        # Sort successors by Manhattan distance (ascending)
        successors.sort(key=lambda x: x[3])  # Sort by the fourth item, which is manhattan_dist
        
        # Add sorted successors to the stack
        for next_state, new_path, new_depth, _ in successors:
            stack.append((next_state, new_path, new_depth))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(current_score)  # Uncomment if backtracking is implemented
    
    return None  # Return None if no solution is found within depth limit